In [17]:
import json

# Load the JSON data from file
with open('data.json') as f:
    data = json.load(f)

# Extract the nodes and links from the JSON data
nodes = data['nodes']
links = data['links']

# Create the physical topology using the nodes and links
physical_topology = {}
for node in nodes:
    physical_topology[node['id']] = []
    for link in links:
        if link['source'] == node['id']:
            physical_topology[node['id']].append(link['target'])
        elif link['target'] == node['id']:
            physical_topology[node['id']].append(link['source'])


# Print the physical and logical topology
print('Physical Topology:', physical_topology)

Physical Topology: {'H1': ['S2', 'S2'], 'H2': ['S1', 'S1'], 'H3': ['S3', 'S3'], 'S1': ['S2', 'S2', 'H2', 'H2'], 'S2': ['S1', 'S1', 'S3', 'S3', 'H1', 'H1'], 'S3': ['S2', 'S2', 'H3', 'H3']}


In [18]:
import networkx as nx


output_dict = {}

for switch, entries in data['flow-tables'].items():
    output_list = []
    for entry in entries:
        match = entry['match']
        actions = entry['actions']
        output_list.append({'Match': match, 'Actions': actions})
    output_dict[switch] = output_list

print(output_dict)

{'S1': [{'Match': {'dl_dst': '01:80:c2:00:00:0e', 'dl_type': 35020}, 'Actions': ['OUTPUT:CONTROLLER']}, {'Match': {'in_port': 1, 'nw_src': '10.0.0.2', 'nw_dst': '10.0.0.1', 'dl_src': '5e:9b:c3:ff:26:dc', 'dl_dst': '82:c8:40:03:27:ef'}, 'Actions': ['OUTPUT:2']}, {'Match': {'in_port': 2, 'nw_src': '10.0.0.1', 'nw_dst': '10.0.0.2', 'dl_src': '82:c8:40:03:27:ef', 'dl_dst': '5e:9b:c3:ff:26:dc'}, 'Actions': ['OUTPUT:1']}, {'Match': {'in_port': 1, 'nw_src': '10.0.0.3', 'nw_dst': '10.0.0.1', 'dl_src': 'a6:52:bd:65:22:ab', 'dl_dst': '82:c8:40:03:27:ef'}, 'Actions': ['OUTPUT:2']}, {'Match': {'in_port': 2, 'nw_src': '10.0.0.1', 'nw_dst': '10.0.0.3', 'dl_src': '82:c8:40:03:27:ef', 'dl_dst': 'a6:52:bd:65:22:ab'}, 'Actions': ['OUTPUT:1']}, {'Match': {}, 'Actions': ['OUTPUT:CONTROLLER']}], 'S2': [{'Match': {'dl_dst': '01:80:c2:00:00:0e', 'dl_type': 35020}, 'Actions': ['OUTPUT:CONTROLLER']}, {'Match': {'in_port': 3, 'nw_src': '10.0.0.2', 'nw_dst': '10.0.0.1', 'dl_src': '5e:9b:c3:ff:26:dc', 'dl_dst': '

In [19]:
parse_data = {}
for i in data['nodes']:
    if i['type'] == 'Host':
        parse_data[i['id']] = i['ip']
parse_data

{'H1': '10.0.0.2', 'H2': '10.0.0.1', 'H3': '10.0.0.3'}

In [20]:
G = nx.Graph()
d = physical_topology
# Add nodes to the graph
for device in d:
    G.add_node(device)

# Add edges to the graph
for device, connections in d.items():
    for connection in connections:
        G.add_edge(device, connection)

# Find all the paths between two nodes
all_paths = []
for node1 in G.nodes():
    for node2 in G.nodes():
        if node1 != node2:
            paths = list(nx.all_simple_paths(G, node1, node2))
            all_paths.extend(paths)



Host = ['H1','H2','H3']
Host_to_Host_Path = []
for path in all_paths:
    if path[0] in Host and path[-1] in Host:
        Host_to_Host_Path.append(path)

print(Host_to_Host_Path)

[['H1', 'S2', 'S1', 'H2'], ['H1', 'S2', 'S3', 'H3'], ['H2', 'S1', 'S2', 'H1'], ['H2', 'S1', 'S2', 'S3', 'H3'], ['H3', 'S3', 'S2', 'H1'], ['H3', 'S3', 'S2', 'S1', 'H2']]


In [21]:
logical_topology = []
for i in Host_to_Host_Path:
    nw_src = parse_data[i[0]]
    nw_dst = parse_data[i[-1]]
    c = 0
    for j in range(1,len(i)-1):
            for k in output_dict[i[j]]:
                try:
                    if nw_src == k['Match']['nw_src'] and nw_dst == k['Match']['nw_dst']:
                        c = c+1
                except:
                    continue
    if c == len(i)-2:
        str1 =  i[0]+'->'+i[-1]
        logical_topology.append(str1)

In [22]:
logical_topology_dict = {'Source':[],
                        'Destination':[]}
for i in logical_topology:
    logical_topology_dict['Source'].append(i[0:2])
    logical_topology_dict['Destination'].append(i[-2:])
logical_topology_dict

{'Source': ['H1', 'H1', 'H2', 'H2', 'H3', 'H3'],
 'Destination': ['H2', 'H3', 'H1', 'H3', 'H1', 'H2']}

In [23]:
import csv
with open('output.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Source', 'Destination'])
    for i in range(len(logical_topology_dict['Source'])):
        writer.writerow([logical_topology_dict['Source'][i], logical_topology_dict['Destination'][i]])